In [1]:
import os
import requests
import datetime
import re

import gspread
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
gc = gspread.service_account(filename="../.keys/shimandou-bot-07299fe8b747.json")

In [3]:
wb = gc.open("stock_list")
ws = wb.worksheet("jp")

In [4]:
df = pd.DataFrame(ws.get_all_records())
print("worksheet loaded")

worksheet loaded


In [5]:
import pandas_datareader.data as pdr
import datetime
end = datetime.date.today()
start = end - datetime.timedelta(days=10)
stock_df = pdr.DataReader("8001.JP", "stooq")

In [281]:
class CompanyProfile():
    def __init__(self, ticker):
        self.ticker = ticker
        self.updated = None
        self.name = None
        self.price = None
        self.pct_change = None
        self.PER = None
        self.PSR = None
        self.PBR = None
        self.ROE = None
        self.ROA = None
        self.dividend_rate = None
        self.market_cap = None
        self.category = None
        self.description = None
        self.url_scouter = f'=HYPERLINK("https://monex.ifis.co.jp/index.php?sa=report_index&bcode={ticker}", "銘")'
        self.url_kabutan = f'=HYPERLINK("https://kabutan.jp/stock/?code={ticker}", "探")'
        self.url_keijiban = None


    def update(self):
        ticker = self.ticker
        yahoo_url = f"https://stocks.finance.yahoo.co.jp/stocks/detail/?code={ticker}.T"
        yahoo_soup = self.get_soup(yahoo_url)

        minkabu_url = f"https://minkabu.jp/stock/{ticker}"
        minkabu_soup = self.get_soup(minkabu_url)

        minkabu_url2 = f"https://minkabu.jp/stock/{ticker}/settlement" #決算情報@みんかぶ
        minkabu_soup2 = self.get_soup(minkabu_url2)

        try:
            self.name = yahoo_soup.select_one(".symbol").text.replace("(株)", "")
        except Exception as e:
            print("not found", e)
            return
        self.updated = datetime.datetime.now().strftime("%Y/%m/%d %H:%M")
        self.price = float(yahoo_soup.select(".stoksPrice")[1].text.replace(",", ""))
        self.pct_change = float(re.findall(r"（(.*)%）", yahoo_soup.select_one(".change").text)[0])
        self.url_keijiban = '=HYPERLINK("{}", "掲")'.format(
            yahoo_soup.select(".subNavi li a")[4]["href"]
        )

        minkabu_page = minkabu_soup.select(".wsnw.fwb")
        self.PER = float(minkabu_page[5].text[:-1].replace(",", "").replace("--", "0"))
        self.PBR = float(minkabu_page[7].text[:-1].replace(",", "").replace("--", "0"))
        self.PSR = float(minkabu_page[6].text[:-1].replace(",", "").replace("--", "0"))
        self.dividend_rate  = float(minkabu_page[3].text[:-1].replace(",", "").replace("--", "0"))
        self.market_cap = int(minkabu_page[9].text[:-3].replace(",", "").replace("--", "0"))
        self.category = minkabu_soup.select(".stock-detail div.ly_content_wrapper.size_ss")[0].select_one("a").text
        self.description = minkabu_soup.select(".stock-detail div.ly_content_wrapper.size_ss")[1].text.strip()


        minkabu_table = minkabu_soup2.select(".data_table")[2].select(".num")
        self.ROA = float(minkabu_table[0].text[:-1].replace("--", "0"))
        self.ROE = float(minkabu_table[2].text[:-1].replace("--", "0"))



    def get_soup(self, url):
        try:
            res = requests.get(url)
            soup = BeautifulSoup(res.text)
            return soup

        except Exception as e:
            print("ERROR: exception occured during requets or soup\n", e)
            return None

    


  

In [282]:
column_convertor = {
    "ticker": "code",
    "dividend_rate": "利回",
    "market_cap": "cap",
    "description": "desc",
    "url_scouter": "銘",
    "url_kabutan": "探",
    "url_keijiban": "掲"
}

In [283]:
UPDATE_ALL = False

In [284]:
for i in df.index:
    ticker = df.loc[i, "code"]
    print(ticker, end="")

    if not(UPDATE_ALL) and df.loc[i, "name"]:
        print("...skipped")
        continue

    company = CompanyProfile(ticker)
    company.update()

    for attr in company.__dict__.keys():
        col = column_convertor[attr] if attr in column_convertor.keys() else attr
        df.loc[i, col] = company.__dict__[attr]
    print("...done")


4490...skipped
8058...skipped
8591...skipped
8267...skipped
2127...skipped
6532...done
3937...done
3985...done
6567...done
3915...done
2769...done
3984...done
6579...done
3031...done
7038...done
3969...done
4442...done
3998...done
3674...done
6195...done
4436...done
6559

AttributeError: 'NoneType' object has no attribute 'text'

In [273]:
list_to_draw = [df.columns.tolist()] + df.fillna("").values.tolist()
ws.update("A1", list_to_draw, value_input_option="USER_ENTERED")

{'spreadsheetId': '1smwmI8bKEPHM3gyz57MKoeK2-OA9c65Ljx3hY8u--SI',
 'updatedRange': 'jp!A1:V52',
 'updatedRows': 52,
 'updatedColumns': 22,
 'updatedCells': 1144}

In [276]:
if df.loc[10, "name"]:
    print("yes")
else:
    print("no")

no
